In [31]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Load and Normalize the Data
---
We will load the data from the included csv file "csgo_round_snapshots.csv".


In [1]:
import pandas as pd
import numpy as np

# percentage of training set to use as validation
valid_size = 0.2


#load the data into a dataframe using pandas
df = pd.read_csv('csgo_round_snapshots.csv')

# obtain training indices that will be used for validation
num_train = len(df)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

#now we must seperate the features from the labels
features = df.drop('round_winner', axis=1)
labels   = df['round_winner']

#perform one hot encoding on the map column
features = pd.get_dummies(features,columns=['map'], drop_first=True)

print(features.head())


#Normalize all the numerical data types 'minmax' 
normalization_func = 'minmax'

for column in features.columns:
    #skips if column is a boolean
    if pd.api.types.is_bool_dtype(features[column]):
        continue

    #skips if column does not change in value
    if features[column].max() - features[column].min() == 0:
        continue
    
    if normalization_func == 'minmax':
        features[column] = (features[column] - features[column].min()) / (features[column].max() - features[column].min())
    elif normalization_func == 'z_score':
        features[column] = (features[column] - features[column].mean()) / features[column].std()
    else:
        continue
    


#shows a brief description of the dataset
features.info()






   time_left  ct_score  t_score  bomb_planted  ct_health  t_health  ct_armor  \
0     175.00       0.0      0.0         False      500.0     500.0       0.0   
1     156.03       0.0      0.0         False      500.0     500.0     400.0   
2      96.03       0.0      0.0         False      391.0     400.0     294.0   
3      76.03       0.0      0.0         False      391.0     400.0     294.0   
4     174.97       1.0      0.0         False      500.0     500.0     192.0   

   t_armor  ct_money  t_money  ...  t_grenade_molotovgrenade  \
0      0.0    4000.0   4000.0  ...                       0.0   
1    300.0     600.0    650.0  ...                       0.0   
2    200.0     750.0    500.0  ...                       0.0   
3    200.0     750.0    500.0  ...                       0.0   
4      0.0   18350.0  10750.0  ...                       0.0   

   ct_grenade_decoygrenade  t_grenade_decoygrenade  map_de_dust2  \
0                      0.0                     0.0          True  

## Define the Architecture

---

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #defining the layers in the NN
        self.output = nn.Linear(102,1)

    
    def forward(self, x):

        x = self.output(x)
        return x


model = Net()
print(model)

#move tensors to GPU if CUDA is available

if train_on_gpu:
    model.cuda()
        

## Specify Loss Function and Optimizer


In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Training the Network


In [ ]:
n_epochs = 100